<a href="https://colab.research.google.com/github/Jonathan5108/COCOObjectDetection/blob/main/COCO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install pycocotools
!pip install requests tqdm

In [ ]:
import os
import requests
from zipfile import ZipFile
from tqdm import tqdm

urls = {
    'train2017': 'http://images.cocodataset.org/zips/train2017.zip',
    'val2017': 'http://images.cocodataset.org/zips/val2017.zip',
    'annotations': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
}
os.makedirs('coco_dataset', exist_ok=True)
os.chdir('coco_dataset')

# Downloads and extracts zip files from the specified url
def download_and_extract(url, dest):
    # Get request from URL
    response = requests.get(url, stream=True)
    # Calculates size of the file
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    # Open and save data to file
    with open(dest, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    # Extract zipfile contents
    with ZipFile(dest, 'r') as zip_ref:
        zip_ref.extractall()
    # Clean up the zipfile after contents extracted
    os.remove(dest)

# Populates our local Colab Environment with the COCO Dataset's training,
# validation, and annotation datasets
for name, url in urls.items():
    download_and_extract(url, f"{name}.zip")

100%|██████████| 19.3G/19.3G [29:11<00:00, 11.0MiB/s]
100%|██████████| 816M/816M [00:48<00:00, 16.7MiB/s]
100%|██████████| 253M/253M [00:24<00:00, 10.4MiB/s]


In [ ]:
import json
import shutil
import random
# Load COCO annotations
with open('/content/coco_dataset/annotations/instances_train2017.json') as f:
    annotations = json.load(f)

# Define what objects from COCO we are looking to detect
categories_of_interest = ['toothbrush', 'scissors', 'mouse']
category_ids = [category['id'] for category in annotations['categories'] if
                category['name'] in categories_of_interest]
# Create directories for the YOLOv8 images and labels datasets
os.makedirs('/content/coco_dataset/yolo_data/images/train', exist_ok=True)
os.makedirs('/content/coco_dataset/yolo_data/images/val', exist_ok=True)
os.makedirs('/content/coco_dataset/yolo_data/labels/train', exist_ok=True)
os.makedirs('/content/coco_dataset/yolo_data/labels/val', exist_ok=True)


In [ ]:
subset_size = 10000 # Increase depending on how much of COCO dataset we want to train on
subset_images = annotations['images'][:subset_size]

# Dataset 80/20 split
random.shuffle(subset_images)
split_index = int(0.8 * len(subset_images))
train_images = subset_images[:split_index]
val_images = subset_images[split_index:]

# Training image annotation conversions
for image in tqdm(train_images):
    image_id = image['id']
    file_name = image['file_name']
    height = image['height']
    width = image['width']
    # Get all relevant annotations in our categories of interest
    image_annotations = [ann for ann in annotations['annotations'] if ann['image_id']
                         == image_id and ann['category_id'] in category_ids]
    if image_annotations:
        # Copy image and labels to appropriate corresponding YOLO directory
        src_img_path = os.path.join('train2017', file_name)
        dest_img_path = os.path.join('/content/coco_dataset/yolo_data/images/train',
                                     file_name)
        shutil.copyfile(src_img_path, dest_img_path)
        label_file_path = os.path.join('/content/coco_dataset/yolo_data/labels/train',
                                       file_name.replace('.jpg', '.txt'))
        # Create YOLO formatted label file from our COCO annotation information
        # (X_Center and Y_Center used instead of X_min and height vars for example)
        with open(label_file_path, 'w') as label_file:
            for annotation in image_annotations:
                bbox = annotation['bbox']
                category_id = annotation['category_id']
                x_center = (bbox[0] + bbox[2]/2) / width
                y_center = (bbox[1] + bbox[3]/2) / height
                w = bbox[2] / width
                h = bbox[3] / height
                label_file.write(f"{category_ids.index(category_id)} {x_center} "
                                f"{y_center} {w} {h}\n")
# Validation image annotation conversions
for image in tqdm(val_images):
    image_id = image['id']
    file_name = image['file_name']
    height = image['height']
    width = image['width']
    # Get all relevant annotations in our categories of interest
    image_annotations = [ann for ann in annotations['annotations'] if ann['image_id']
                         == image_id and ann['category_id'] in category_ids]
    if image_annotations:
      # Copy image and labels to appropriate corresponding YOLO directory
        src_img_path = os.path.join('train2017', file_name)
        dest_img_path = os.path.join('/content/coco_dataset/yolo_data/images/val',
                                     file_name)
        shutil.copyfile(src_img_path, dest_img_path)
        label_file_path = os.path.join('/content/coco_dataset/yolo_data/labels/val',
                                       file_name.replace('.jpg', '.txt'))
        # Create YOLO formatted label file from our COCO annotation information
        # (X_Center and Y_Center used instead of X_min and height vars for example)
        with open(label_file_path, 'w') as label_file:
            for annotation in image_annotations:
                bbox = annotation['bbox']
                category_id = annotation['category_id']
                x_center = (bbox[0] + bbox[2]/2) / width
                y_center = (bbox[1] + bbox[3]/2) / height
                w = bbox[2] / width
                h = bbox[3] / height
                label_file.write(f"{category_ids.index(category_id)} {x_center} "
                                f"{y_center} {w} {h}\n")

100%|██████████| 2000/2000 [05:43<00:00,  5.82it/s]


Write to the COCO YAML configuration file with 3 classes (for our 3 objects) and the specified paths for our training and validation image datasets

In [ ]:
# COCO YAML Configuration File
coco_yaml_content = """
train: ./yolo_data/images/train
val: ./yolo_data/images/val

nc: 3
names: ['mouse', 'scissors', 'toothbrush']
"""

with open("/content/coco_dataset/coco.yaml", "w") as file:
    file.write(coco_yaml_content)

Model Training (Epochs to be determined) Using YOLOv8n model

In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# Train model (More epochs can be done at expense of computational power)
model.train(data='/content/coco_dataset/coco.yaml',
            epochs=100,
            imgsz=640,
            batch=16,
            name='yolov8_coco',
            workers=4)


Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/coco_dataset/coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=yolov8_coco2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /content/coco_dataset/yolo_data/labels/train... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:00<00:00, 1350.44it/s]

train: New cache created: /content/coco_dataset/yolo_data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/coco_dataset/yolo_data/labels/val... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<00:00, 1029.77it/s]

val: New cache created: /content/coco_dataset/yolo_data/labels/val.cache


...

                   all        120        181      0.925      0.739      0.829      0.645



100 epochs completed in 0.135 hours.
Optimizer stripped from runs/detect/yolov8_coco2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolov8_coco2/weights/best.pt, 6.2MB

Validating runs/detect/yolov8_coco2/weights/best.pt...
Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.45it/s]


                   all        120        181      0.915      0.741      0.836       0.65
                 mouse         55         76       0.91      0.829      0.901      0.691
              scissors         34         48      0.976      0.862      0.942      0.812
            toothbrush         31         57      0.859      0.533      0.664      0.448
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/yolov8_coco2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d79a5f00730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

Create zipped file containing model w/ weights

In [1]:
import shutil
from google.colab import files

shutil.make_archive('yolov8_coco', 'zip', '/content/runs/train/yolov8_coco')
files.download('yolov8_coco.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Visualize
def plot_boxes(img, boxes, confidences, class_ids, model_names):
    counts = {}

    for box, conf, cls in zip(boxes, confidences, class_ids): # Iterate using zip
        x1, y1, x2, y2 = box
        cls_name = model_names[int(cls)]
        if cls_name in counts:
            counts[cls_name] += 1
        else:
            counts[cls_name] = 1

        label = f'{model.names[int(cls)]} {counts[cls_name]} {conf:.2f}'
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    2, (0, 255, 0), 2)

    y_offset = 30
    for cls_name, count in counts.items():
        cv2.putText(img, f'{cls_name}: {count}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)
        y_offset += 20

    return img

# Path to the image
image_paths = ['/content/scissorsmice.jpg', '/content/scissorsopen.jpg']
for image_path in image_paths:
    image = Image.open(image_path)
    image = np.array(image)

    # Resulting bounding boxes predicted by model
    results = model(image_path)

    if results[0].boxes.xyxy is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()  #.cpu for CPU usage, remove otherwise
        confidences = results[0].boxes.conf.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy()
    else:
        boxes = []
        confidences = []
        class_ids = []

    # Plot the image with bounding boxes
    img_with_boxes = plot_boxes(image.copy(), boxes, confidences, class_ids, model.names) # Pass
    # confidences and class IDs

    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(img_with_boxes)
    plt.axis('off')
    plt.show()

ModuleNotFoundError: No module named 'ultralytics'